In [1]:
# Importing Libraries
from heat_pump_model import heat_pump
from libraries import * 
import pandas as pd
import numpy as np  
import plotly.graph_objects as go

# Providing the Temperature inputs for the COP
cop_temp_dict = {
    'COP 6': [95, 51.1],
    'COP 5': [95, 43.3],
    'COP 4': [95, 32.8],
    'COP 3': [95, 19.5],
    'COP 2': [112.8, 15],
}

# Providing the fixed natural gas prices
ng_prices = []
for i in range(18,101):ng_prices.append(i/10)

columns = ['COP 2', 'COP 3', 'COP 4', 'COP 5', 'COP 6']
df = pd.DataFrame(0.0, index = range(len(ng_prices)), columns = columns)
df['NG price'] = ng_prices


In [2]:
hp = heat_pump()
hp.hot_temperature_minimum = 90
hp.process_heat_requirement = [0,1]*12 * 365

#hp.carbon_price_per_ton = 20.0

for cop in columns:
    hp.hot_temperature_desired = cop_temp_dict[cop][0]
    hp.cold_temperature_available = cop_temp_dict[cop][1]

    for i in df.index:
        hp.gas_price_MMBTU = [df.loc[i,'NG price']]*8760
        #print(df.loc[i,'NG price'])

        delta = 1.0
        while abs(delta) > 0.1:
            # Calc loop
            hp.calculate_COP()
            hp.calculate_energy_and_mass_flow()
            hp.calculate_heat_pump_costs()
            hp.calculate_natural_gas_comparison()
            
            delta = hp.gas_LCOH - hp.LCOH
            if delta > 0:
                current_price = sum(hp.utility_rate_kWh)/8760
                hp.utility_rate_kWh = [current_price+0.001] * 8760
            else:
                current_price = sum(hp.utility_rate_kWh)/8760
                hp.utility_rate_kWh = [current_price - 0.001] * 8760

        #print(delta)
        current_price = sum(hp.utility_rate_kWh)/8760
        #print(current_price)
        df.loc[i,cop] = current_price

Costs: $ 42152.75
Gas Lifetime LCOH:  9.627  $/MMBTU
Refrigerant selection should be further analyzed before implementation.

Potential refigerants for this process include:  ['Ammonia', 'Butane', 'Ethanol', 'Methanol', 'R11', 'R113', 'R114', 'R12', 'R123', 'R1233zd(E)', 'R1234ze(E)', 'R1234ze(Z)', 'R124', 'R13I1', 'R141b', 'R142b', 'R152A', 'R21', 'R236EA', 'R236FA', 'R245ca', 'R245fa', 'R365MFC', 'R40', 'RC318', 'n-Pentane', 'Water']

The recommended refrigerant for this process is:  R218
Warning this refrigerant was selected for its low critical pressure, 
all refrigerants should be examined in a detailed analysis.

The refrigerant previously selected went super critical or did not meet 
other parameters, for the purposes of the COP calculation the refrigerant 
will be replaced with R245ca.
Estimated Compressor Efficiency: 0.9

Calculate COP Called
Theoretical COP:  8.38
Estimated COP:  5.62
Calculate Energy and Mass Called
Hot Mass Flow Average:  17.877  kg/s
Cold Average Outlet Te

In [3]:
print(df)

    COP 2  COP 3  COP 4  COP 5  COP 6  NG price
0   0.007  0.017  0.026  0.036  0.046       1.8
1   0.009  0.018  0.027  0.037  0.047       1.9
2   0.010  0.020  0.029  0.039  0.049       2.0
3   0.011  0.021  0.031  0.042  0.052       2.1
4   0.012  0.023  0.034  0.044  0.055       2.2
..    ...    ...    ...    ...    ...       ...
78  0.089  0.139  0.187  0.237  0.286       9.6
79  0.090  0.140  0.189  0.239  0.289       9.7
80  0.091  0.142  0.192  0.242  0.292       9.8
81  0.092  0.143  0.194  0.245  0.295       9.9
82  0.093  0.145  0.196  0.247  0.298      10.0

[83 rows x 6 columns]


In [4]:

fig = go.Figure()

for cop in columns:
    fig.add_trace(go.Scatter(
        x=df['NG price'], 
        y=df[cop],
        mode='lines',
        fill='tonexty',
        name=cop
    ))
fig.update_layout(
    template = 'simple_white',
    title = 'Break-Even kWh Price based on COP',
    xaxis_title = 'Price of Natural Gas ($/MMBTU)',
    yaxis_title = 'Electricity Price ($/kWh)'
    )
fig.show()

In [5]:

hp.refrigerant = 'R1234ze(Z)'

print(hp.internal_rate_of_return)
hp.utility_rate_kwh=[0.02]*8760
hp.calculate_COP()
hp.calculate_energy_and_mass_flow()
hp.calculate_heat_pump_costs()
hp.calculate_natural_gas_comparison()
hp.calculate_cash_flow()
print(hp.internal_rate_of_return)

-1.0
Refrigerant selection should be further analyzed before implementation.

Potential refigerants for this process include:  ['Ammonia', 'Butane', 'Ethanol', 'Methanol', 'R11', 'R113', 'R114', 'R12', 'R123', 'R1233zd(E)', 'R1234ze(E)', 'R1234ze(Z)', 'R124', 'R13I1', 'R141b', 'R142b', 'R152A', 'R21', 'R236EA', 'R236FA', 'R245ca', 'R245fa', 'R365MFC', 'R40', 'RC318', 'n-Pentane', 'Water']

The recommended refrigerant for this process is:  R218
Warning this refrigerant was selected for its low critical pressure, 
all refrigerants should be examined in a detailed analysis.

The refrigerant previously selected went super critical or did not meet 
other parameters, for the purposes of the COP calculation the refrigerant 
will be replaced with R245ca.
Estimated Compressor Efficiency: 0.9

Calculate COP Called
Theoretical COP:  8.38
Estimated COP:  5.62
Calculate Energy and Mass Called
Hot Mass Flow Average:  17.877  kg/s
Cold Average Outlet Temperature:  29.56 °C
Average Power Draw of Heat 